## more practice with cnn with pytorch
## try running this ec2 server see if it would be faster


In [1]:
#imports
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [2]:


BATCH = 32
EPOCHS = 2
LR = 0.001

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# try torch.cude.get_device_name(device_id)
# torch.get_device_name(device_id)


In [14]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1050 Ti with Max-Q Design'

In [20]:
torch.cuda.current_device()

0

In [3]:
train_df = pd.read_csv('data/train.csv', dtype=np.float32)
test_df = pd.read_csv('data/test.csv', dtype=np.float32)
sample_sub = pd.read_csv('data/sample_submission.csv')

In [4]:
print(train_df.shape)

train_df.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
Y_Train = train_df.label.values
X0 = train_df.loc[:, train_df.columns != 'label'].values/255

X0.shape, Y_Train.shape


Y_Train = torch.from_numpy(Y_Train).type(torch.LongTensor)


In [6]:


X_Train_1 = X0.reshape((-1, 1, 28, 28))
X_Train_1.shape


(42000, 1, 28, 28)

In [7]:


X_Train = np.stack((X_Train_1,)*3, axis=1)
X_Train = np.squeeze(X_Train, axis=2)
X_Train.shape



(42000, 3, 28, 28)

In [8]:

Transform = transforms.Compose(
    [transforms.ToTensor()])


In [9]:


class GetData(Dataset):
    def __init__(self, X_Train, Y_Train, Transform):
        self.X = X_Train
        self.transform = Transform
        self.Y = Y_Train

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.transform(self.X[index]).permute((1, 2, 0)).contiguous(), self.Y[index]


In [10]:

trainset = GetData(X_Train, Y_Train, Transform)
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True, num_workers=4)



In [11]:

model = torchvision.models.resnet152()
model.fc = nn.Linear(2048, 10, bias=True)


In [12]:


model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


In [13]:


for epoch in range(EPOCHS):
    tr_loss = 0.0

    model = model.train()

    for i, (images, labels) in enumerate(trainloader):

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss += loss.detach().item()

    model.eval()
    print('Epoch: %d | Loss: %.4f'%(epoch, tr_loss / i))


KeyboardInterrupt: 

In [ ]:


Xt_1 = (test_df.values/255).reshape((-1, 1, 28, 28))
Xt_1.shape


In [ ]:
Xt = np.stack((Xt_1,)*3, axis=1)
Xt = np.squeeze(Xt, axis=2)
Xt.shape

In [ ]:


with torch.no_grad():
    model.eval()
    sample_sub['Label'] = model(torch.from_numpy(Xt).to(DEVICE)).cpu().argmax(dim=1)


In [ ]:


sample_sub.head()

